# Conceitos avançados

* ### Prompt Few-shot
* #### É uma forma de ensinar o modelo a base de exemplos, mostrando o padrão de resposta, melhorando a formatação e precisão da resposta.

* É similar a formatação de mensagens da api da OpenAI, mas com uma sintaxe diferente: 

```python
    mensagens = [
        {'role': 'user','content': 'quanto é 1+1'}, 
        {'role': 'assistant','content': '2'},
        {'role': 'user','content': 'quanto é 10 * 5?' },
        {'role': 'assistant', 'content':'50'},
        {'role': 'user','content': 'quanto é 10 + 3?' },
        
    ]

In [4]:
#import libs 

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
import os

#carrega arquivo .env ao ambiente 
load_dotenv()

if not os.getenv('GOOGLE_API_KEY'): 
    raise ValueError("Chave GOOGLE_API_KEY NÃO encontrada no arquivo .env")

chat = ChatGoogleGenerativeAI(model='gemini-1.5-flash')


In [5]:
mensagens = [
    HumanMessage(content='quanto é 1 + 1?'),
    AIMessage(content='2'),
    HumanMessage(content='quanto é 10 * 5?'),
    AIMessage(content='50'),
    HumanMessage(content='quanto é 10 + 3?')
]

resposta = chat.invoke(mensagens)

print(resposta.content)

13


In [6]:
#debugar 

import langchain

langchain.debug = True
chat.invoke(mensagens)

langchain.debug = False



[llm/start] [llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human: quanto é 1 + 1?\nAI: 2\nHuman: quanto é 10 * 5?\nAI: 50\nHuman: quanto é 10 + 3?"
  ]
}
[llm/end] [llm:ChatGoogleGenerativeAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "13",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": []
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "13",
            "response_metadata": {
              "prompt_feedback": {
                "block_reason": 0,
                "safety_ratings": []
              },
              "finish_reason": "STOP",
              "safety_ratings": []
            },
            "type": "ai",
            "id": "ru

## Caching

- **Cache de memória** - serve para economziar tokens, viusto que ele irá salvar respostas em cache, caso tenha a mesma pergunta ele pega do cache

In [7]:
#import libs

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [8]:
mensagens = [
    SystemMessage(content='Você é um assistente engraçado.'),
    HumanMessage(content='Quanto é 1 + 1? ')

]

In [9]:
# cacheamento

from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

set_llm_cache(InMemoryCache())

In [10]:
%%time

chat.invoke(mensagens)

CPU times: total: 0 ns
Wall time: 1.37 s


AIMessage(content='Depende! Se estivermos falando de números inteiros em um sistema numérico padrão, a resposta é, obviamente, 2.  Mas e se estivermos falando de maçãs?  Então você teria duas maçãs deliciosas, prontas para uma receita incrível de torta! Ou talvez dois problemas se você não conseguir decidir qual comer primeiro.  E se for amor?  Ah, amor... 1 + 1 é igual a um amor infinitamente maior, cheio de beijos, abraços e muito, muito mais do que duas pessoas separadas!  A matemática é divertida, não é? 😉', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--7c0bc866-a862-4133-8876-3d2f7cd9c4e3-0', usage_metadata={'input_tokens': 15, 'output_tokens': 125, 'total_tokens': 140, 'input_token_details': {'cache_read': 0}})

In [11]:
%%time

chat.invoke(mensagens)

CPU times: total: 0 ns
Wall time: 1.07 ms


AIMessage(content='Depende! Se estivermos falando de números inteiros em um sistema numérico padrão, a resposta é, obviamente, 2.  Mas e se estivermos falando de maçãs?  Então você teria duas maçãs deliciosas, prontas para uma receita incrível de torta! Ou talvez dois problemas se você não conseguir decidir qual comer primeiro.  E se for amor?  Ah, amor... 1 + 1 é igual a um amor infinitamente maior, cheio de beijos, abraços e muito, muito mais do que duas pessoas separadas!  A matemática é divertida, não é? 😉', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--7c0bc866-a862-4133-8876-3d2f7cd9c4e3-0', usage_metadata={'input_tokens': 15, 'output_tokens': 125, 'total_tokens': 140, 'input_token_details': {'cache_read': 0}, 'total_cost': 0})

## Caching

- **Cache com SQLite** - forma de cache em banco, mesmo desligando a aplicação os dados ainda ficam salvos

In [12]:
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

# 3. Configure o cache com o caminho corrigido
set_llm_cache(SQLiteCache(database_path='Arquivos/Langchain_cache.sqlite'))


In [25]:
%%time

chat.invoke(mensagens)

CPU times: total: 0 ns
Wall time: 2.39 ms


AIMessage(content='Depende.  Você quer a resposta matemática chata e previsível, ou a resposta divertida e imprevisível que envolve viagens no tempo, a natureza da realidade e a possível existência de universos paralelos onde 1 + 1 = abacate?  \n\n...Estou brincando (a menos que...?). A resposta é 2.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--df59e492-fbb1-4cc4-ad14-c5552ec05627-0', usage_metadata={'input_tokens': 15, 'output_tokens': 72, 'total_tokens': 87, 'input_token_details': {'cache_read': 0}, 'total_cost': 0})

In [17]:
%%time

chat.invoke(mensagens)

CPU times: total: 0 ns
Wall time: 2.24 ms


AIMessage(content='Depende.  Você quer a resposta matemática chata e previsível, ou a resposta divertida e imprevisível que envolve viagens no tempo, a natureza da realidade e a possível existência de universos paralelos onde 1 + 1 = abacate?  \n\n...Estou brincando (a menos que...?). A resposta é 2.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--df59e492-fbb1-4cc4-ad14-c5552ec05627-0', usage_metadata={'input_tokens': 15, 'output_tokens': 72, 'total_tokens': 87, 'input_token_details': {'cache_read': 0}, 'total_cost': 0})